In [1]:
import os
import json
import math

# 세 점의 좌표 cor1, cor2, cor3에 대하여 각 cor2의 값을 구하는 함수. (코사인 제2 법칙 사용)
def _cal_angle(cor1, cor2, cor3):
    dist1 = math.sqrt( (cor1[0] - cor2[0])**2 + (cor1[1] - cor2[1])**2 + (cor1[2] - cor2[2])**2 )
    dist2 = math.sqrt( (cor3[0] - cor2[0])**2 + (cor3[1] - cor2[1])**2 + (cor3[2] - cor2[2])**2 )
    dist3 = math.sqrt( (cor1[0] - cor3[0])**2 + (cor1[1] - cor3[1])**2 + (cor1[2] - cor3[2])**2 )
    return math.acos((dist1**2 + dist2**2 - dist3**2)/(2*dist1*dist2))

# 세 관절의 좌표 리스트 cors1, cors2, cors3에 대하여 각 각도의 평균을 구하는 함수.
def cal_angle(cors1, cors2, cors3):
    # cors = [(x0, y0, z0), ...]
    val = 0
    for i in range(len(cors1)):
        val += _cal_angle(cors1[i], cors2[i], cors3[i])
    return val / len(cors1)

# 허리 각을 구할 때, 왼쪽 무릎과 오른쪽 무릎 중 큰 각을 택하여 평균을 내기 위한 함수.
# 디디고 있는 다리의 각을 허리각으로 재며, 이 각이 항상 크다.
def cal_angle2(cors1, cors2, cors3, cors4):
    # cors = [(x0, y0, z0), ...]
    val = 0
    for i in range(len(cors1)):
        val += max(_cal_angle(cors1[i], cors2[i], cors3[i]), _cal_angle(cors1[i], cors2[i], cors4[i]))
    return val / len(cors1)

# 어깨의 높이차의 평균을 구하기 위한 함수.
def gap_y(cors1, cors2):
    val = 0
    for i in range(len(cors1)):
        val += abs(cors1[i][1] - cors2[i][1])
    return val / len(cors1)

# 원하는 관절의 좌표 리스트를 구하는 함수.
def choose_data(data, idx1, idx2):
    # data = [(x0, y0, z0), (x1, y1, z1), ... ]
    li = []
    for i in idx1:
        li.append((data["annotations"][i]["keypoints"][idx2], data["annotations"][i]["keypoints"][idx2+1], data["annotations"][i]["keypoints"][idx2+2]))
    return li

data_path = 'C:\\Users\\jh.yun\\Desktop\\anno_3D.tar_'
target_path = 'C:\\Users\\jh.yun\\Desktop\\target_data'

os.chdir(data_path)
data_list = os.listdir()

# 카운팅 할 사람 수
person_num = 0

for i in data_list:
    with open(i, "r", encoding='utf8') as f:
        data = json.loads(f.read())
        
        # 각 카메라당 사람 별 데이터 구분
        person_dict = {}
        for j in data["annotations"]:
            if str(j["person_no"]) not in person_dict.keys():
                person_dict[str(j["person_no"])] = []
            person_dict[str(j["person_no"])].append(data["annotations"].index(j))
            
        os.chdir(target_path)
        
        # 관절 좌표 처리 및 저장
        for j in person_dict.keys():
            person_num += 1
            head_cors = choose_data(data, person_dict[j], 54)
            neck_cors = choose_data(data, person_dict[j], 48)
            chest_cors = choose_data(data, person_dict[j], 42)
            v1 = cal_angle(head_cors, neck_cors, chest_cors)
            
            right_shoulder_cors = choose_data(data, person_dict[j], 72)
            left_shoulder_cors = choose_data(data, person_dict[j], 78)
            v2 = gap_y(right_shoulder_cors, left_shoulder_cors)
        
            peivis_cors = choose_data(data, person_dict[j], 36)
            right_hip_cors = choose_data(data, person_dict[j], 12)
            left_hip_cors = choose_data(data, person_dict[j], 18)
        
            right_knee_cors = choose_data(data, person_dict[j], 6)
            left_knee_cors = choose_data(data, person_dict[j], 24)
            v3 = cal_angle2(chest_cors, peivis_cors, right_knee_cors, left_knee_cors)
            v4 = cal_angle(right_knee_cors, peivis_cors, left_knee_cors)
        
            right_ankle_cors = choose_data(data, person_dict[j], 0)
            left_ankle_cors = choose_data(data, person_dict[j], 30)
            v5 = cal_angle(right_hip_cors, right_knee_cors, right_ankle_cors)
            v6 = cal_angle(left_hip_cors, left_knee_cors, right_ankle_cors)
            
            f = open(str(person_num)+".txt", mode='wt', encoding='utf8')
            f.write(str(v1)+"-"+str(v2)+"-"+str(v3)+"-"+str(v4)+"-"+str(v5)+"-"+str(v6)+"\n")
            f.close()
        os.chdir(data_path)